# Locally simultaneous inference for the "most promising" effects

We study the problem of constructing confidence intervals for the "most promising" effects. We consider two instantiations of the problem: inference on the winner and the file-drawer problem. The first provides a confidence interval for the mean of the maximum observation, and the latter provides a confidence region for the mean of all observations that exceed a pre-specified threshold. We compare conditional inference due to Lee et al. [1], hybrid inference due to Andrews et al. [2], simultaneous inference via max-z confidence intervals, and locally simultaneous inference.

[1] Lee, J. D., Sun, D. L., Sun, Y., & Taylor, J. E. (2016). Exact post-selection inference, with application to the lasso. Annals of Statistics 44(3), 907-927.

[2] Andrews, I., Kitagawa, T., & McCloskey, A. (2019). Inference on winners. Forthcoming at Quarterly Journal of Economics.

In [ ]:
%load_ext autoreload
%autoreload 2
from scipy.optimize import linprog
import numpy as np
from numpy.linalg import inv
from sklearn import linear_model
import scipy
import math
import matplotlib.pyplot as plt

from methods import *
from utils import *

# Inference on the winner

We construct a confidence interval for the mean of the largest entry of $y$, where $y\sim \mathcal{N}(\mu,I)$.

In [ ]:
def iow_trial(mu_vec, Sigma, alpha, nu, beta, plausible_gap, SI_halfwidth):
# one trial of constructing intervals for the winner
    m = len(mu_vec)
    y = mu_vec + np.random.normal(size=m)

    selected_ind = np.argmax(y)

    # simultaneous intervals
    SI_int = [y[selected_ind] - SI_halfwidth, y[selected_ind] + SI_halfwidth]

    # locally simultaneous intervals
    plausible_inds = plausible_winners(y, plausible_gap)
    LSI_int = locally_simultaneous_inference(y[selected_ind], Sigma, plausible_inds, [selected_ind], alpha=alpha, nu=nu)

    # conditional intervals
    A, b = inference_on_winner_polyhedron(m, selected_ind)
    eta = np.zeros(m)
    eta[selected_ind] = 1
    cond_int = conditional_inference(y, Sigma, A, b, eta, alpha=alpha)

     # hybrid intervals
    hybrid_int = hybrid_inference(y, Sigma, A, b, eta, alpha=alpha, beta=beta)

    return SI_int, LSI_int, cond_int, hybrid_int

In [ ]:
# simulation params
trials = 100
ms = np.array([10**j for j in range(1,4)])

alpha = 0.1
nu = 0.01 # for LSI
beta = 0.01 # for hybrid

Cs = [10, 30, 50]
thetas = [0.5, 1, 2, 4]


plausible_gaps = np.zeros(len(ms))
SI_halfwidths = np.zeros(len(ms))
for i in range(len(ms)):
    m = ms[i]
    plausible_gaps[i] = 4*max_z_width(np.eye(m), nu)
    SI_halfwidths[i] = max_z_width(np.eye(m), alpha)


for C in Cs:
    for theta in thetas:
        m = 10
        mu_vec0 = np.abs(np.array(range(1,m+1)) - 0.5*(m+1))**theta
        mu_vec0 = mu_vec0 - np.min(mu_vec0)
        norm_const = np.max(mu_vec0)

        cond_widths = np.zeros((len(ms), trials))
        hybrid_widths = np.zeros((len(ms), trials))
        local_widths = np.zeros((len(ms), trials))
        simultaneous_widths = np.zeros((len(ms), trials))

        for i in range(len(ms)):
            m = round(ms[i])
            Sigma = np.eye(m)

            mu_vec = np.abs(np.array(range(1,m+1)) - 0.5*(m+1))**theta
            mu_vec = -C/norm_const*(mu_vec - np.min(mu_vec))

            for j in range(trials):
                print("Trial {}, C {}, theta {}, m {}".format(j,C,theta,m))
                SI_int, LSI_int, cond_int, hybrid_int = iow_trial(mu_vec, Sigma, alpha, nu, beta, plausible_gaps[i], SI_halfwidths[i])
                simultaneous_widths[i, j] = SI_int[1] - SI_int[0]
                local_widths[i, j] = LSI_int[1] - LSI_int[0]
                cond_widths[i, j] = cond_int[1] - cond_int[0]
                hybrid_widths[i, j] = hybrid_int[1] - hybrid_int[0]

        
        plot_title = 'ϴ = ' + str(theta) + ', C = ' + str(C)
        ylabel = 'interval width'
        xlabel = 'm'
        filename = 'IOW_theta' + str(theta) + 'trials' + str(trials) + 'C' + str(C) + '.pdf'
        plot_and_save_results(ms, simultaneous_widths, local_widths, cond_widths, hybrid_widths, plot_title, ylabel, xlabel, filename, alpha=0.1, log_scale=True, plot_baseline=True)

# Filedrawer problem

We construct a confidence region for the mean of the all entries of $y$ that exceed a given threshold $T$, where $y\sim \mathcal{N}(\mu,\Sigma)$ and $\Sigma$ follows the RBF kernel.

In [ ]:
def filedrawer_trial(mu_vec, Sigma, T, alpha, nu, beta, plausible_gap, SI_halfwidth, compute_conditional=False):
# one trial of constructing intervals for the winner
    m = len(mu_vec)
    y = mu_vec + np.random.multivariate_normal(np.zeros(m), Sigma)

    selected_inds = np.where(y > T)[0]
    
    if len(selected_inds) == 0:
        return [0,0], [0,0], [0,0], [0,0]
    
    selected_sigmas = np.sqrt(np.diag(Sigma)[selected_inds])
    
    # simultaneous intervals
    SI_ints = [y[selected_inds] - SI_halfwidth*selected_sigmas, y[selected_inds] + SI_halfwidth*selected_sigmas]
    
    # locally simultaneous intervals
    plausible_inds = plausible_filedrawer(y, plausible_gap, T)
    LSI_ints = locally_simultaneous_inference(y[selected_inds], Sigma, plausible_inds, selected_inds, alpha=alpha, nu=nu)

    # conditional intervals
    A, b = filedrawer_polyhedron(m, selected_inds, T)
    num_selected = len(selected_inds)
    cond_ints = [np.zeros(len(selected_inds)), np.zeros(len(selected_inds))]
    hybrid_ints = [np.zeros(len(selected_inds)), np.zeros(len(selected_inds))]
    for i in range(len(selected_inds)):
        eta = np.zeros(m)
        eta[selected_inds[i]] = 1
        if compute_conditional:
            [cond_ints[0][i], cond_ints[1][i]] = conditional_inference(y, Sigma, A, b, eta, alpha=alpha/num_selected)
        [hybrid_ints[0][i], hybrid_ints[1][i]] = hybrid_inference(y, Sigma, A, b, eta, alpha=alpha/num_selected, beta=beta)

    return SI_ints, LSI_ints, cond_ints, hybrid_ints

In [ ]:
# simulation params
trials = 100
scales = np.linspace(1,9,5)
T = -1 # threshold for selection

alpha = 0.1
nu = 0.01 # for LSI
beta = 0.01 # for hybrid

Cs = [10, 30, 50]
thetas = [0.5, 1, 2, 4]
m = 100

compute_conditional = False

plausible_gaps = np.zeros(len(scales))
SI_halfwidths = np.zeros(len(scales))
for i in range(len(scales)):
    scale = scales[i]
    z = np.reshape(range(m),(m,1))
    Sigma = exponentiated_quadratic(z, z, scale)
    plausible_gaps[i] = 2*max_z_width(Sigma, nu)
    SI_halfwidths[i] = max_z_width(Sigma, alpha)

for C in Cs:
    for theta in thetas:
        
        cond_widths = np.zeros((len(scales), trials))
        hybrid_widths = np.zeros((len(scales), trials))
        local_widths = np.zeros((len(scales), trials))
        simultaneous_widths = np.zeros((len(scales), trials))
        
        for i in range(len(scales)):
            scale = scales[i]
            z = np.reshape(range(m),(m,1))
            Sigma = exponentiated_quadratic(z, z, scale)

            mu_vec0 = np.abs(np.array(range(1,m+1)) - 0.5*(m+1))**theta
            mu_vec0 = mu_vec0 - np.min(mu_vec0)
            norm_const = np.max(mu_vec0)
            mu_vec = np.abs(np.array(range(1,m+1)) - 0.5*(m+1))**theta
            mu_vec = -C/norm_const*(mu_vec - np.min(mu_vec))

            for j in range(trials):
                print("Trial {}, scale {}, C {}, theta {}".format(j,scales[i], C, theta))
                SI_ints, LSI_ints, cond_ints, hybrid_ints = filedrawer_trial(mu_vec, Sigma, T, alpha, nu, beta, plausible_gaps[i], SI_halfwidths[i], compute_conditional=compute_conditional)
                simultaneous_widths[i, j] = 2*SI_halfwidths[i]*1 # simultaneous intervals are of fixed width
                local_widths[i, j] = np.mean(LSI_ints[1] - LSI_ints[0])
                if compute_conditional:
                    cond_widths[i, j] = np.mean(cond_ints[1] - cond_ints[0])
                hybrid_widths[i, j] = np.mean(hybrid_ints[1] - hybrid_ints[0])

                
        plot_title = 'ϴ = ' + str(theta) + ', C = ' + str(C)
        ylabel = 'interval width'
        xlabel = 'kernel scale'
        filename = 'Filedrawer_theta' + str(theta) + 'trials' + str(trials) + 'C' + str(C) + '.pdf'
        plot_and_save_results(scales, simultaneous_widths, local_widths, cond_widths, hybrid_widths, plot_title, ylabel, xlabel, filename, plot_conditional=compute_conditional, plot_baseline=True)

## Inference on the maximum of two Gaussians
Reproduces the experiment from Figure 1 in the paper.

In [ ]:
trials = 100
Deltas = np.linspace(0,20,21)
alpha = 0.1
nu = 0.005

cond_widths = np.zeros((len(Deltas), trials))
local_widths = np.zeros((len(Deltas), trials))
simultaneous_widths = np.zeros((len(Deltas), trials))


m = 2
Sigma = np.eye(m)


simultaneous_widths[:,:] = 2*max_z_width(Sigma, alpha)
    
for i in range(len(Deltas)):
    print("Delta = {}".format(Deltas[i]))
    mu_vec = np.zeros(2)
    mu_vec[0] = Deltas[i]

    for j in range(trials):

        y = mu_vec + np.random.normal(size=2)

        selected_ind = np.argmax(y)

        eta = np.zeros(m)
        eta[selected_ind] = 1
        

        # locally simultaneous intervals
        gap = 2*np.sqrt(2)*scipy.stats.norm.isf(nu/2)
        if np.abs(y[1]-y[0]) < gap:
            plausible_winners = 2
        else:
            plausible_winners = 1


        if plausible_winners == 2:
            local_widths[i,j] = simultaneous_widths[0,0]
        else:
            local_widths[i,j] = 2*scipy.stats.norm.isf((1-nu)*alpha/2)

        # conditional intervals
        A, b = inference_on_winner_polyhedron(m, selected_ind)

        cond_int = conditional_inference(y, Sigma, A, b, eta, alpha=alpha)
        cond_widths[i,j] = cond_int[1] - cond_int[0]


plot_title = None
ylabel = 'interval width'
xlabel = 'Δ'
filename = 'intro_comparison.pdf'
plot_and_save_results(Deltas, simultaneous_widths, local_widths, cond_widths, hybrid_widths, plot_title, ylabel, xlabel, filename, plot_hybrid=False, plot_baseline=False)

# Inference on the winner with nonparametric intervals

We construct a confidence interval for the mean of the largest entry of $\bar y$, where $\bar y = \frac 1 n \sum_{i=1}^n y_i$, for i.i.d. $y_i = \mu + \xi_i$. Here, $\mu$ is a fixed vector and $\xi_i$ has independent entries drawn from a beta distribution. We apply conditional inference and hybrid inference with a normal approximation. Simultaneous inference and locally simultaneous inference use betting-based confidence intervals of Waudby-Smith and Ramdas (2023).

In [ ]:
def iow_trial_nonparametric(mu_vec, n, beta_a, beta_b, alpha, nu, beta):
# one trial of constructing intervals for the winner
    m = len(mu_vec)
    noise_mat = scipy.stats.beta.rvs(beta_a, beta_b, size=(m,n))
    noise_mean = np.reshape(np.mean(noise_mat, axis = 1), (m,1))
    noise_std = np.reshape(np.std(noise_mat,axis = 1), (m,1))
    
    y = mu_vec + noise_mat
    y_normed = (np.divide(mu_vec + noise_mean,noise_std)*np.sqrt(n)).squeeze()
    
    selected_ind = np.argmax(y_normed)
    
    # simultaneous interval
    selected_ind_samples = y[selected_ind,:]
    lb = mu_vec[selected_ind]
    ub = mu_vec[selected_ind]+2
    selected_ind_samples = (selected_ind_samples-lb)/(ub-lb)   
    SI_int = wsr(selected_ind_samples, alpha/m)
    SI_int[0] = SI_int[0]*(ub-lb)+lb
    SI_int[1] = SI_int[1]*(ub-lb)+lb
    
    # locally simultaneous interval
    plausible_gap = 4*bentkus_gap(n, nu/m)
    plausible_number = np.sum(mu_vec + noise_mean > np.max(mu_vec + noise_mean) - plausible_gap) # used for Bonferroni
    LSI_int = wsr(selected_ind_samples, (alpha-nu)/plausible_number)
    LSI_int[0] = LSI_int[0]*(ub-lb)+lb
    LSI_int[1] = LSI_int[1]*(ub-lb)+lb
    
    # conditional interval
    eta = np.zeros(m)
    eta[selected_ind] = 1
    A, b = inference_on_winner_polyhedron(m, selected_ind)
    
    
    cond_int = conditional_inference(y_normed, np.eye(m), A, b, eta, alpha=alpha)
    cond_int[0] = cond_int[0]/np.sqrt(n)*noise_std[selected_ind]
    cond_int[1] = cond_int[1]/np.sqrt(n)*noise_std[selected_ind]
    
    # hybrid interval
    hybrid_int = hybrid_inference(y_normed, np.eye(m), A, b, eta, alpha=alpha, beta=beta)
    hybrid_int[0] = hybrid_int[0]/np.sqrt(n)*noise_std[selected_ind]
    hybrid_int[1] = hybrid_int[1]/np.sqrt(n)*noise_std[selected_ind]

    return SI_int, LSI_int, cond_int, hybrid_int, selected_ind

In [ ]:
# simulation params
ns = np.linspace(20,100,5)
m = 100
trials = 100
alpha = 0.1
nu = 0.01
beta = 0.01
C = 20
thetas = [0.5, 1, 2, 4]

# beta parameters
beta_as = [0.7, 0.8, 0.9]
    
for beta_a in beta_as:
    
    beta_b = 1-beta_a
    beta_mean = beta_a/(beta_a + beta_b)
    
    for theta in thetas:

        cond_coverage = np.zeros((len(ns),1))
        local_coverage = np.zeros((len(ns),1))
        hybrid_coverage = np.zeros((len(ns),1))
        simultaneous_coverage = np.zeros((len(ns),1))

        cond_widths = np.zeros((len(ns), trials))
        hybrid_widths = np.zeros((len(ns), trials))
        local_widths = np.zeros((len(ns), trials))
        simultaneous_widths = np.zeros((len(ns), trials))


        for i in range(len(ns)):

            n = round(ns[i])
            noise_samples = n
            
            mu_vec = np.abs(np.array(range(1,m+1)) - 0.5*(m+1))**theta
            mu_vec = mu_vec - np.min(mu_vec)
            mu_vec /= np.max(mu_vec)
            mu_vec *= -C
            mu_vec = np.reshape(mu_vec, (m,1))
            

    
            for j in range(trials):
                print("Trial {}, theta {}, sample size {}".format(j, theta, n))
                SI_int, LSI_int, cond_int, hybrid_int, selected_ind = iow_trial_nonparametric(mu_vec, n, beta_a, beta_b, alpha, nu, beta)
                
                true_target = mu_vec[selected_ind] + beta_mean
                
                simultaneous_widths[i, j] = SI_int[1] - SI_int[0]
                simultaneous_coverage[i] += (SI_int[1] >= true_target)*(SI_int[0] <= true_target)/trials
                local_widths[i, j] = LSI_int[1] - LSI_int[0]
                local_coverage[i] += (LSI_int[1] >= true_target)*(LSI_int[0] <= true_target)/trials
                cond_widths[i, j] = cond_int[1] - cond_int[0]
                cond_coverage[i] += (cond_int[1] >= true_target)*(cond_int[0] <= true_target)/trials
                hybrid_widths[i, j] = hybrid_int[1] - hybrid_int[0]
                hybrid_coverage[i] += (hybrid_int[1] >= true_target)*(hybrid_int[0] <= true_target)/trials
                
         
        

        plot_title = 'ϴ = ' + str(theta)  + ', a = ' + str(beta_a) + ', b = ' + str("{:.1f}".format(beta_b))
        ylabel = 'coverage'
        xlabel = 'n'
        filename = 'IOW_np_coverage_theta' + str(theta) + 'trials' + str(trials) + 'a' + str(beta_a) + '.pdf'
        plot_and_save_results(ns, simultaneous_coverage, local_coverage, cond_coverage, hybrid_coverage, plot_title, ylabel, xlabel, filename, fill_between=False, plot_baseline=True, baseline_val=0.9, ylim = [0, 1.05])
        ylabel = 'interval width'
        filename = 'IOW_np_theta' + str(theta) + 'trials' + str(trials) + 'a' + str(beta_a) + '.pdf'
        plot_and_save_results(ns, simultaneous_widths, local_widths, cond_widths, hybrid_widths, plot_title, ylabel, xlabel, filename)